# **Anime Recommender System**

In [ ]:
#pip install surprise

### **Importing Dependecies**

In [ ]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from surprise import Reader, Dataset, 

### **Loading Data**

In [ ]:
# Data 1

anime_interactions_df = pd.read_csv("/content/rating.csv")

data = Dataset.load_from_df(anime_interactions_df[['user_id', 'anime_id', 'rating']], Reader)

In [ ]:
import pandas as pd

file_path = '/content/main_pred.csv'

columns_to_use = ['anime_id', 'name', 'genre', 'type', 'episodes', 'rating_x']

anime_data = pd.read_csv(file_path, usecols=columns_to_use)



In [ ]:
anime_data.head()

,anime_id,name,genre,type,episodes,rating_x
0,32281,Kimi no Na wa.,"drama,romance,school,supernatural",Movie,1,9.37
1,5114,Fullmetal Alchemist: Brotherhood,"action,adventure,drama,fantasy,magic,military,...",TV,64,9.26
2,28977,Gintama°,"action,comedy,historical,parody,samurai,sci-fi...",TV,51,9.25
3,9253,Steins;Gate,"sci-fi,thriller",TV,24,9.17
4,9969,Gintama&#039;,"action,comedy,historical,parody,samurai,sci-fi...",TV,51,9.16


In [ ]:
anime_data['genre'] = anime_data['genre'].str.split(',')

anime_data_expanded = anime_data.explode('genre')

genre_counts = anime_data_expanded['genre'].value_counts()

print(genre_counts)

comedy           2475
action           1714
adventure        1199
fantasy          1168
sci-fi           1143
drama            1129
shounen          1127
romance          1091
school            827
supernatural      713
sliceoflife       650
mecha             509
ecchi             493
magic             484
hentai            371
shoujo            359
seinen            357
mystery           347
historical        312
superpower        311
military          278
sports            277
harem             270
kids              263
music             234
space             207
horror            203
parody            196
martialarts       163
psychological     161
demons            158
police            127
game              109
samurai            84
vampire            77
thriller           70
shoujoai           49
shounenai          48
josei              43
dementia           33
cars               33
yaoi               32
yuri               17
Name: genre, dtype: int64


In [ ]:
anime_data['genre'].value_counts()

[hentai]                                                  266
[comedy]                                                  132
[music]                                                    61
[comedy, sliceoflife]                                      59
[comedy, school, sliceoflife]                              51
                                                         ... 
[action, kids, mecha, sports]                               1
[action, horror, supernatural, vampire]                     1
[comedy, ecchi, harem, romance, school, sci-fi]             1
[comedy, drama, fantasy, mystery, shounen, superpower]      1
[action, horror, sci-fi, supernatural, vampire, yaoi]       1
Name: genre, Length: 2299, dtype: int64

### **Functions**

In [ ]:
def get_genre():
    # Prompt user for desired genre
    genre = input("Enter your preferred genre: ")
    return genre

def get_type():
    # Provide options for anime type
    type_options = ["Movie", 'TV', 'OVA', 'Special', 'Movie' ,'ONA', 'Music' ]

    # Prompt user for desired type
    while True:
        type_choice = input("Choose your preferred type: ")

        if type_choice in type_options:
            return type_choice
        else:
            print("Invalid choice. Please select a valid option.")

def get_duration():
    # Provide options for anime duration
    duration_options = ["Short", "Medium", "Long"]

    # Prompt user for desired duration
    while True:
        duration_choice = input("Choose your preferred duration (Short/Medium/Long): ")

        if duration_choice in duration_options:
            return duration_choice
        else:
            print("Invalid choice. Please select a valid option.")


In [ ]:

user_genre = get_genre()
user_type = get_type()
user_duration = get_duration()


Enter your preferred genre: game,psychological,horror,mystery
Choose your preferred type: Movie
Choose your preferred duration (Short/Medium/Long): Short


### **Data preprocessing**

In [ ]:
tfidf_vectorizer = TfidfVectorizer()
anime_data["genre_vector"] = list(tfidf_vectorizer.fit_transform(anime_data["genre"].astype(str)).toarray())


In [ ]:
user_genre_vector = tfidf_vectorizer.transform([user_genre]).toarray()

filtered_anime = anime_data[anime_data["type"] == user_type]


### **Collaborative Filtering (CF)**

In [ ]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(anime_interactions_df[['user_id', 'anime_id', 'rating']], reader)

### **Model Training**

In [ ]:
#model = SVD(n_factors=100)
#model.fit(data.build_full_trainset())

In [ ]:
cf_predictions = []

for anime_id in filtered_anime["anime_id"]:
    prediction = model.predict(uid=1, iid=anime_id)
    cf_predictions.append(prediction)

In [ ]:
genre_similarity = cosine_similarity(user_genre_vector, list(filtered_anime["genre_vector"]))


In [ ]:
hybrid_scores = genre_similarity.flatten() + [prediction.est for prediction in cf_predictions]

filtered_anime["hybrid_scores"] = hybrid_scores


<ipython-input-33-1483428a1c6d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_anime["hybrid_scores"] = hybrid_scores


### **Recommendation**

In [ ]:
top_k = 5
recommended_anime = filtered_anime.sort_values(by="hybrid_scores", ascending=False).head(top_k)

# Print recommendations
print("Recommended Anime:")
for i in range(len(recommended_anime)):
    print(f"{i+1}. {recommended_anime.iloc[i]['name']}")

Recommended Anime:
1. Ookami Kodomo no Ame to Yuki
2. Suzumiya Haruhi no Shoushitsu
3. Neon Genesis Evangelion: The End of Evangelion
4. Acchi Kocchi
5. Accel World: Infinite∞Burst
